https://keras.io/preprocessing/image/

# TensorFlow 2.0

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Hyperparameter Tunning

In [2]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.7

input_shape = (32, 32, 3)
num_classes = 10

## Preprocess

In [3]:
import pandas as pd

In [4]:
train_df = pd.read_csv('train_dataset.csv')
test_df = pd.read_csv('test_dataset.csv')

In [5]:
train_df.head()

,path,class_name
0,../../dataset/cifar/cifar/train\0_frog.png,frog
1,../../dataset/cifar/cifar/train\10000_automobi...,automobile
2,../../dataset/cifar/cifar/train\10001_frog.png,frog
3,../../dataset/cifar/cifar/train\10002_frog.png,frog
4,../../dataset/cifar/cifar/train\10003_ship.png,ship


In [6]:
test_df.head()

,path,class_name
0,../../dataset/cifar/cifar/test\0_cat.png,cat
1,../../dataset/cifar/cifar/test\1000_dog.png,dog
2,../../dataset/cifar/cifar/test\1001_airplane.png,airplane
3,../../dataset/cifar/cifar/test\1002_ship.png,ship
4,../../dataset/cifar/cifar/test\1003_deer.png,deer


### 1. train_datagen

In [7]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        width_shift_range=0.3,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

### 2. train_generator
- train_datagen.flow_from_dataframe( 이젠 path말고, df넣기 )

In [8]:
train_generator = train_datagen.flow_from_dataframe(
        train_df,
        x_col='path',
        y_col='class_name',
        target_size=input_shape[:2],
        batch_size=batch_size
)

validation_generator = test_datagen.flow_from_dataframe(
        test_df,
        x_col='path',
        y_col='class_name',
        target_size=input_shape[:2],
        batch_size=batch_size
)

Found 50000 validated image filenames belonging to 10 classes.


## Build Model

In [10]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

W0903 12:58:46.338918 31752 nn_ops.py:4283] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0903 12:58:46.384796 31752 nn_ops.py:4283] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0903 12:58:46.424690 31752 nn_ops.py:4283] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


In [11]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

## Training

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=num_epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator))

W0903 12:58:46.667042 31752 nn_ops.py:4283] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0903 12:58:46.689981 31752 nn_ops.py:4283] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0903 12:58:46.726882 31752 deprecation.py:323] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\ops\math_grad.py:1394: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
1563/1563 [==============================] - 253s 162ms/step - loss: 1.8713 - accuracy: 0.3015 - val_loss: 1.6046 - val_accuracy: 0.4245
Epoch 2/10
  91/1563 [>.............................] - ETA: 3:48 - loss: 1.6961 - accuracy: 0.3784